In [1]:
print("all ok")

all ok


In [3]:
print("all ok")

all ok


In [2]:
from dotenv import load_dotenv 
import os

load_dotenv()

True

In [3]:
# Here we first import the class ChatOpenAI from the langchain_openai package. This class is used to create an instance of a chat model that can interact with users in a conversational manner. By importing this class, we can utilize its functionalities to build chat-based applications or interfaces that leverage the capabilities of OpenAI's language models.
from langchain_openai import ChatOpenAI

In [4]:
# Create an object of the class ChatOpenAI and assign it to the variable chat. This object will allow us to interact with the OpenAI chat model and utilize its features for generating responses in a conversational context.
chat_llm = ChatOpenAI(model="gpt-4o-mini")

In [8]:
chat_llm.invoke("Hello, how are you?").content

"Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?"